# ISLR Chapter 5 Resampling Methods

## Lab Exercises

In [13]:
'''
references:
JWarmenhoven: https://github.com/JWarmenhoven/ISLR-python/blob/master/Notebooks/
collinprather: https://github.com/collinprather/ISLR-Python
Jliu: https://github.com/0liu/ISLR
http://www.science.smith.edu/~jcrouser/SDS293/labs/lab7-py.html
'''

# Math and data processing
import numpy as np
import scipy as sp
import pandas as pd

# scikit-learn
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.utils import resample
from sklearn.preprocessing import scale, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Visulization
from IPython.display import display
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn-white')

## 5.3.1 Validation Set Approach

In [14]:
auto = pd.read_csv('Auto.csv', na_values='?').dropna()
auto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 396
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight        392 non-null    int64  
 5   acceleration  392 non-null    float64
 6   year          392 non-null    int64  
 7   origin        392 non-null    int64  
 8   name          392 non-null    object 
dtypes: float64(4), int64(4), object(1)
memory usage: 30.6+ KB


In [3]:
auto

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
392,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl
393,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup
394,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage
395,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger


This is the method to split data employed by jcrouser 

In [15]:
# alternative method to split data into test train sets, total rows is 392, so by sampling 196 points we create 50:50 split
train_df = auto.sample(196, random_state = 1)
test_df = auto[~auto.isin(train_df)].dropna(how = 'all')

X_train = train_df['horsepower'].values.reshape(-1,1)
y_train = train_df['mpg']
X_test = test_df['horsepower'].values.reshape(-1,1)
y_test = test_df['mpg']

This is the method to split data used by jliu

In [16]:
# Simple linear regression features and response
features = ['horsepower']
response = ['mpg']
X = auto[features]
y = auto[response]

# Split Auto data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=196, random_state=47) #using sklearn built-in functions

# Regression
auto_slr = LinearRegression()
auto_slr.fit(X_train, y_train)

# Prediction and MSE
y_pred = auto_slr.predict(X_test)
print("SLR MSE = ", mean_squared_error(y_test, y_pred))

SLR MSE =  25.273723992970933


The estimated test MSE for the linear regression fit is 23.36. We can use the  PolynomialFeatures()  function to estimate the test error for the polynomial and cubic regressions. PolynomialFeatures() generate a new feature matrix consisting of all polynomial combinations of the features with degree less than or equal to the specified degree.

In [5]:
# Polynomial regression features of degree 2
poly2 = PolynomialFeatures(degree=2)
X2 = poly2.fit_transform(X)

# Split Auto data into train and test sets
X2_train, X2_test, y_train, y_test = train_test_split(X2, y, test_size=196, random_state=47)
a
# Regression
auto_poly2 = LinearRegression()
auto_poly2.fit(X2_train, y_train)

# Prediction and MSE
y2_pred = auto_poly2.predict(X2_test)
print("Polynomial regression of degree 2: MSE = ", mean_squared_error(y_test, y2_pred))

Polynomial regression of degree 2: MSE =  18.869003119495538


In [7]:
# Polynomial regression features of degree 3
poly3 = PolynomialFeatures(degree=3)
X3 = poly3.fit_transform(X)

# Split Auto data into train and test sets
X3_train, X3_test, y_train, y_test = train_test_split(X3, y, test_size=196, random_state=47)

# Regression
auto_poly3 = LinearRegression()
auto_poly3.fit(X3_train, y_train)

# Prediction and MSE
y3_pred = auto_poly3.predict(X3_test)
print("Polynomial regression of degree 3: MSE = ", mean_squared_error(y_test, y3_pred))

Polynomial regression of degree 3: MSE =  18.833366995930167


These results are consistent with our previous findings: a model that predicts  mpg  using a quadratic function of  horsepower  performs better than a model that involves only a linear function of  horsepower , and there is little evidence in favor of a model that uses a cubic function of  horsepower .

## 5.3.2 Leave-One-Out Cross-Validation

In [8]:
# Polynomial regression over degrees from 1 (simple linear) to 5
auto_poly = LinearRegression()
loocv = LeaveOneOut()

for poly_deg in range(1, 6):
    print("\nPolynomial regression of degree {}:".format(poly_deg))
    
    poly = PolynomialFeatures(degree=poly_deg)
    X_d = poly.fit_transform(X)
    scores = cross_val_score(auto_poly, X_d, y, cv=loocv, scoring='neg_mean_squared_error')
    
    loocv_mse = scores.mean() * (-1)  # sign-flip to convert score to MSE
    print('  MSE = {}\n'.format(loocv_mse))


Polynomial regression of degree 1:
  MSE = 24.231513517929226


Polynomial regression of degree 2:
  MSE = 19.248213124489396


Polynomial regression of degree 3:
  MSE = 19.334984064133813


Polynomial regression of degree 4:
  MSE = 19.424430309411886


Polynomial regression of degree 5:
  MSE = 19.033211842978396



We see a sharp drop in the estimated test MSE between the linear and quadratic fits, but then no clear improvement from using higher-order polynomials.

## 5.3.3 k-Fold Cross-Validation

The KFold function can (intuitively) also be used to implement k-fold CV. Below we use k = 10, a common choice for k.

In [10]:
# Polynomial regression over degrees from 1 (simple linear) to 10
auto_poly = LinearRegression()

kfold = KFold(n_splits=10, random_state=47, shuffle=False)

for poly_deg in range(1, 11):
    print("\nPolynomial regression of degree {}:".format(poly_deg))
    
    poly = PolynomialFeatures(degree=poly_deg)
    X_d = poly.fit_transform(X)
    scores = cross_val_score(auto_poly, X_d, y, cv=kfold, scoring='neg_mean_squared_error')
    
    loocv_mse = scores.mean() * (-1)  # sign-flip to convert score to MSE
    print('  MSE = {}\n'.format(loocv_mse))

/home/ggbong/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning



Polynomial regression of degree 1:
  MSE = 27.439933652339864


Polynomial regression of degree 2:
  MSE = 21.23584005580211


Polynomial regression of degree 3:
  MSE = 21.336606183328694


Polynomial regression of degree 4:
  MSE = 21.353886994209773


Polynomial regression of degree 5:
  MSE = 20.905646119059934


Polynomial regression of degree 6:
  MSE = 20.82189095906726


Polynomial regression of degree 7:
  MSE = 20.953534894379217


Polynomial regression of degree 8:
  MSE = 21.077131510426256


Polynomial regression of degree 9:
  MSE = 21.03675183384266


Polynomial regression of degree 10:
  MSE = 20.981013741561554



Notice that the computation time is much shorter than that of LOOCV. (In principle, the computation time for LOOCV for a least squares linear model should be faster than for k-fold CV, due to the availability of the formula (5.2) for LOOCV; however, unfortunately the KFold() function does not make use of this formula.) We still see little evidence that using cubic or higher-order polynomial terms leads to lower test error than simply using a quadratic fit.

## 5.3.4 The Bootstrap

In [25]:
portfolio = pd.read_csv('Portfolio.csv')
portfolio

,X,Y
0,-0.895251,-0.234924
1,-1.562454,-0.885176
2,-0.417090,0.271888
3,1.044356,-0.734198
4,-0.315568,0.841983
...,...,...
95,0.479091,1.454774
96,-0.535020,-0.399175
97,-0.773129,-0.957175
98,0.403634,1.396038


In [26]:
# Function to calculate the alpha for portfolio allocation
def alpha(data):
    """
    data: pandas dataframe with two columns X and Y.
    """

    sigma = data.cov()  # covariance matrix
    var_x = sigma.X['X'] # select column X and row X
    var_y = sigma.Y['Y']
    cov_xy = sigma.X['Y']
    alpha = (var_y - cov_xy) / (var_x + var_y - 2 * cov_xy)
    
    return alpha

alpha_original = alpha(portfolio)
print("Portfolio alpha = ", alpha_original)

Portfolio alpha =  0.57583207459283


In [32]:
# Bootstrap with B=1000 on portfolio data
N = portfolio.shape[0]
B = 1000 #bootstrap samples

portfolio_b = resample(portfolio, n_samples=N*B, random_state=42) # create 1000 (B) bootstrap samples of size N each
alphas = [alpha(group) for name, group in portfolio_b.groupby(np.arange(N * B) // N)] # group by every 1000 points to generate a list of 1000 alphas

alpha_bias = np.mean(alphas) - alpha_original #bias compared to actual
alpha_se = np.std(alphas)  #error of alpha

alpha_bootstrap = pd.DataFrame([[alpha_original, alpha_bias, alpha_se],],
                              columns=['original', 'bias', 'std. error'], index=['alpha'])

display(alpha_bootstrap)

,original,bias,std. error
alpha,0.575832,0.001963,0.089929


Continuing with jliu bootstrap method, need to **create a function that extracts linear regression coefficient from each bootstrap sample**

In [16]:
# Function to get simple linear regression coefficients for Auto data set
def auto_coef(data, features, response):
    """
    data: pandas dataframe sampled from the Auto data set
    features: a string list of feature names
    response: a string of response names
    """

    auto_reg = LinearRegression()
    auto_reg.fit(data[features], data[response])
    return [auto_reg.intercept_] + list(auto_reg.coef_)

features = ['horsepower']
response = 'mpg'
coef_original = pd.DataFrame([auto_coef(auto, features, response)], columns=['Intercept'] + features, index=[''])
print("\nmpg ~ horsepower coefficients:\n\n", coef_original)


mpg ~ horsepower coefficients:

   Intercept  horsepower
  39.935861   -0.157845


In [17]:
# Bootstrap with B=1000 on Auto data
N = auto.shape[0]
B = 1000
auto_b = resample(auto, n_samples=N*B, random_state=42)
coefs = [auto_coef(group, features, response) for name, group in auto_b.groupby(np.arange(N * B) // N)]
coefs_df = pd.DataFrame(coefs, columns=['Intercept'] + features)
coef_bias = coefs_df.mean() - coef_original
coef_se = coefs_df.std()
coef_bootstrap = pd.concat([coef_original.T.copy(), coef_bias.T, coef_se], axis=1)
coef_bootstrap.columns = ['original', 'bias', 'std. error']
display(coef_bootstrap)

,original,bias,std. error
Intercept,39.935861,0.033521,0.869087
horsepower,-0.157845,-0.000500,0.007503


In [25]:
auto = pd.read_csv('Auto.csv', na_values='?').dropna()

In [28]:
lm = LinearRegression()
X = auto['horsepower'].values.reshape(-1,1)
y = auto['mpg']
clf = lm.fit(X,y)
print(clf.coef_, clf.intercept_)

[-0.15784473] 39.93586102117047


In [29]:
Xsamp, ysamp = resample(X, y, n_samples=1000)
clf = lm.fit(Xsamp,ysamp)
print('Intercept: ' + str(clf.intercept_) + " Coef: " + str(clf.coef_))

Intercept: 38.73686745681643 Coef: [-0.14596684]


In [31]:
Xsamp.shape

(1000, 1)

## Alternative approach to Bootstrap by jrouser (incomplete)

In [22]:
portfolio_df = pd.read_csv('Portfolio.csv')
display(portfolio_df)

,X,Y
0,-0.895251,-0.234924
1,-1.562454,-0.885176
2,-0.417090,0.271888
3,1.044356,-0.734198
4,-0.315568,0.841983
...,...,...
95,0.479091,1.454774
96,-0.535020,-0.399175
97,-0.773129,-0.957175
98,0.403634,1.396038


In [18]:
def alpha(X,Y):
    return ((np.var(Y)-np.cov(X,Y))/(np.var(X)+np.var(Y)-2*np.cov(X,Y)))

Estimating $\alpha$ using all 100 observations:

In [19]:
X = portfolio_df.X[0:100]
y = portfolio_df.Y[0:100]
print(alpha(X,y))

[[1.07270947 0.57665115]
 [0.57665115 0.06414064]]


The next command uses the sample() function to randomly select 100 observations from the range 1 to 100, with replacement. This is equivalent to constructing **one** new bootstrap data set and recomputing  $\alpha$  based on the new data set.

In [ ]:
dfsample = portfolio_df.sample(frac=1, replace=True)
X = dfsample.X[0:100]
y = dfsample.Y[0:100]
print(alpha(X,y))

In [20]:
def bstrap(df):
    tresult = 0
    for i in range(0,1000): #create 1000 bootstrap samples
        dfsample = df.sample(frac=1, replace=True) #frac=1 means for each bootstrap, sample the same number as the dataset
        X = dfsample.X[0:100]
        y = dfsample.Y[0:100]
        result = alpha(X,y)
        tresult += result
    fresult = tresult / 1000
    print(fresult)
    
bstrap(portfolio_df) # returns the average of the 1000 bootstrap samples

[[1.12314555 0.58054718]
 [0.58054718 0.09624484]]


Incomplete.....

## Conceptual Exercises

3a) k-fold cross-validation is implemented by taking the set of n observations and randomly splitting into k non-overlapping groups. Each of these groups acts as a validation set and the remainder as a training set. The test error is estimated by averaging the k resulting MSE estimates.

3bi) Compared to validation set approach where half of training data is turned to validation set, k-fold cross-validation provides better model fit as all the data is used for training. In other words, 1) the estimated test error rate can be high depending on which data is in the training set. 2) The validation set error may overestimate the test error rate for the model fit on the entire data set

3bii) Compared to Leave One Out Cross validation (LOOCV), k-fold CV requires much less computation power as the model is only fit k times, typically between 5 and 10, compared to n times when LOOCV is used. Also, LOOCV has higher variance but lower bias than k-fold CV.

4)To estimate the standard deviation of our prediction, we use the bootstrap method (sampling with replacement), where we create many bootstrap samples and for each one we fit a model and compute our response Y prediction. We then take the standard deviation of the many Y predictions. 